In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
comstock_buildings = ['quickservicerestaurant',
                      'fullservicerestaurant',
                      'smalloffice',
                      'mediumoffice',
                      'largeoffice',
                      'warehouse',
                      'smallhotel',
                      'largehotel',
                      'outpatient',
                      'hospital',
                      'secondaryschool',
                      'primaryschool',
                      'retailstandalone',
                      'retailstripmall']

test_comstock_buildings = ['smalloffice']

In [3]:
upgrades = ['5','6','7','8','9','10','15','17','18']
test_upgrades = ['17']

In [4]:
# loop over upgrades
for upgrade in upgrades:
    UP = str(upgrade)
    up = f"{int(upgrade):02}"  # Convert string to int and format with leading zero
        
    ##loop over buildings
    #for building in test_comstock_buildings:

    directory = f'annual/data/{up}'
    file_path = f'{directory}/{up}_data.csv'
    df = pd.read_csv(file_path)

    # Load the U.S. States Shapefile
    gdf_states = gpd.read_file('cb_2018_us_state_500k/cb_2018_us_state_500k.shp')

    # create savings_df for each building
    # Group by 'in.comstock_building_type'
    grouped = df.groupby('comstock_building_type')

    # Iterate over each building group, filter columns that match the defined columns, and save as separate files
    for building_type, group in grouped:
        # Merge GeoDataFrame with building type grouped df based on state abbreviations
        gdf_merged = gdf_states.merge(group, left_on='STUSPS', right_on='state')

        # Filter out non-continental states (Alaska, Hawaii, and territories)
        gdf_continental = gdf_merged[~gdf_merged['STUSPS'].isin(['AK', 'HI', 'PR', 'GU', 'VI', 'MP', 'AS'])]

        # Plotting percentage_above_25
        fig, ax = plt.subplots(1, 1, figsize=(15, 10))
        gdf_continental.plot(column='percentage_above_25', ax=ax, legend=True, cmap='coolwarm',
                             legend_kwds={'label': 'Percentage of Buildings w savings > 25%',
                                          'orientation': "horizontal"})
        ax.set_title(f"{up} {building_type} Percentage of Buildings w savings > 25% by State - Continental US")
        ax.set_axis_off()  # Remove axis and tick labels
        #plt.show()
        directory = f'annual/plots/states/{up}'
        os.makedirs(directory, exist_ok=True)  # Ensure the directory exists
        plt.savefig(f"{directory}/{up}_{building_type}_savings_25.png", bbox_inches='tight', dpi=300)
        plt.close(fig)

        # Plotting perc_gas_savings
        fig, ax = plt.subplots(1, 1, figsize=(15, 10))
        gdf_continental.plot(column='percentage_above_50', ax=ax, legend=True, cmap='coolwarm',
                             legend_kwds={'label': 'Percentage of Buildings w savings > 50%',
                                          'orientation': "horizontal"})
        ax.set_title(f"{up} {building_type} Percentage of Buildings w savings > 50% by State - Continental US")
        ax.set_axis_off()  # Remove axis and tick labels
        #plt.show()
        plt.savefig(f"{directory}/{up}_{building_type}_savings_50.png", bbox_inches='tight', dpi=300)
        plt.close(fig)
